# Advent of Code 2024

In [27]:
import re
from collections import Counter, defaultdict
from dataclasses import dataclass, field
from heapq import heappop, heappush
from itertools import combinations

import numpy as np
import pandas as pd

In [3]:
from enum import Enum


class Direction(Enum):
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3


def rotate_right(direction):
    return Direction((direction.value + 1) % 4)


def rotate_left(direction):
    return Direction((direction.value - 1) % 4)


def is_opposite(d1, d2):
    return rotate_right(rotate_right(d1)) == d2


def get_direction(symbol):
    return Direction("^>v<".index(symbol.lower()))


dx_vectors = np.array(
    [
        [-1, 0],
        [0, 1],
        [1, 0],
        [0, -1],
    ]
)


def get_dx(direction):
    return dx_vectors[direction.value]


#### 1.1

In [53]:
data = np.loadtxt("inputs/1.txt", dtype=int)
sorted_data = np.sort(data, axis=0)
np.abs(sorted_data[:, 0] - sorted_data[:, 1]).sum()

np.int64(2000468)

#### 1.2

In [54]:
counts = pd.concat(
    [
        pd.Series(sorted_data[:, 0]).value_counts().rename("0"),
        pd.Series(sorted_data[:, 1]).value_counts().rename("1"),
    ],
    axis=1,
).dropna()

(counts.iloc[:, 0] * counts.iloc[:, 1] * counts.index).sum()

np.float64(18567089.0)

#### 2.1

In [55]:
reports = [np.fromiter(line.split(), int) for line in open("inputs/2.txt")]

In [56]:
def is_safe(report):
    changes = np.diff(report)
    signs = np.sign(changes)
    return signs[0] != 0 and (signs == signs[0]).all() and (np.abs(changes) <= 3).all()


sum(map(is_safe, reports))

np.int64(502)

#### 2.2

In [57]:
def are_valid_changes(changes, possible_sign):
    return (np.sign(changes) == possible_sign) & (np.abs(changes) <= 3)


def is_safe_with_dampener(report):
    if report.size < 3:
        return True

    changes = np.diff(report)
    for possible_sign in [-1, 1]:
        errors = ~are_valid_changes(changes, possible_sign)
        n_errors = errors.sum()
        if n_errors > 2:
            continue
        elif n_errors == 2:
            idcs = np.nonzero(errors)[0]
            if idcs[0] + 1 != idcs[1] or not are_valid_changes(changes[idcs[0]] + changes[idcs[1]], possible_sign):
                continue
        elif n_errors == 1:
            idx = np.argmax(errors)
            if (
                idx != 0
                and idx != changes.size - 1
                and not are_valid_changes(changes[idx] + changes[idx + 1], possible_sign)
                and not are_valid_changes(changes[idx - 1] + changes[idx], possible_sign)
            ):
                continue
        return True
    return False


sum(map(is_safe_with_dampener, reports))

544

#### 3.1

In [58]:
memory = open("inputs/3.txt").read()

In [59]:
sum(int(a) * int(b) for a, b in re.findall(r"mul\(([0-9]+),([0-9]+)\)", memory))

180233229

#### 3.2

In [60]:
def calc_with_conditionals(memory):
    total_sum = 0
    enabled = True
    for a, b, do_string, dont_string in re.findall(r"mul\(([0-9]+),([0-9]+)\)|(do\(\))|(don't\(\))", memory):
        if a:
            if enabled:
                total_sum += int(a) * int(b)
        elif do_string:
            enabled = True
        elif dont_string:
            enabled = False
    return total_sum


calc_with_conditionals(memory)

95411583

#### 4.1

In [61]:
txt = np.array(list(map(list, open("inputs/4.txt").read().split())))

In [62]:
def count_xmas(text, words=("XMAS", "SAMX")):
    total_count = 0
    n, m = text.shape

    for view in [
        text,
        text.T,
        [np.diagonal(text, offset=offset) for offset in range(-n + 1, m)],
        [np.diagonal(np.flipud(text), offset=offset) for offset in range(-n + 1, m)],
    ]:
        total_count += sum("".join(line).count(word) for line in view for word in words)

    return total_count


count_xmas(txt)

2530

#### 4.2

In [63]:
def count_max(text):
    total_count = 0
    n, m = text.shape
    for i_min in range(n - 2):
        for j_min in range(m - 2):
            if text[i_min + 1, j_min + 1] == "A":
                i_max = i_min + 2
                j_max = j_min + 2
                diag_txt = text[[i_min, i_min, i_max, i_max], [j_min, j_max, j_min, j_max]]
                if diag_txt[0] != diag_txt[3] and np.count_nonzero(diag_txt == "S") == 2 and np.count_nonzero(diag_txt == "M") == 2:
                    total_count += 1
    return total_count


count_max(txt)

1921

#### 5.1

In [64]:
rules_txt, updates_txt = open("inputs/5.txt").read().split("\n\n")
rules = [(int(x), int(y)) for x, y in (rule.split("|") for rule in rules_txt.split())]
updates = [[int(x) for x in line.split(",")] for line in updates_txt.split()]
succeeding_pages = defaultdict(set)
for rule in rules:
    succeeding_pages[rule[0]].add(rule[1])

In [65]:
def is_correctly_ordered(update):
    return not any(x in succeeding_pages[y] for x, y in combinations(update, 2))


def middle_page(update):
    return update[(len(update) - 1) // 2]


total_sum = sum(middle_page(update) for update in updates if is_correctly_ordered(update))
print(total_sum)

6384


#### 5.2

In [66]:
from copy import copy


def fix_update(update):
    update = copy(update)
    for i, _ in enumerate(update):
        j = i + 1
        while j < len(update):
            if update[i] in succeeding_pages[update[j]]:
                update[i], update[j] = update[j], update[i]
                j = i + 1
            else:
                j += 1
    return update


incorrect_updates = (update for update in updates if not is_correctly_ordered(update))
sum(middle_page(fix_update(update)) for update in incorrect_updates)


5353

#### 6.1

In [67]:
class Grid:
    def __init__(self, grid_txt):
        self.grid = np.array(list(map(list, grid_txt.split())))
        self.n, self.m = self.grid.shape
        self.reset_history()

    def reset_history(self):
        self.history = np.zeros((*self.grid.shape, 4), bool)

    def get(self, pos):
        return self.grid[tuple(pos)]

    def move_forward(self, pos, direction):
        return pos + get_dx(direction)

    def next_state(self, pos, direction):
        new_pos = self.move_forward(pos, direction)
        if self.out_of_bounds(new_pos):
            return new_pos, direction, True
        if self.get(new_pos) == "#":
            direction = rotate_right(direction)
        else:
            pos = new_pos
        return pos, direction, False

    def find_guard_position(self):
        for i in range(self.n):
            for j in range(self.m):
                if self.grid[i, j] != "." and self.grid[i, j] != "#":
                    return i, j
        return -1, -1

    def out_of_bounds(self, pos):
        return pos[0] < 0 or pos[0] >= self.n or pos[1] < 0 or pos[1] >= self.m

    def mark_visited(self, pos, direction):
        self.history[pos[0], pos[1], direction.value] = True

    def already_visited(self, pos, direction):
        return self.history[pos[0], pos[1], direction.value]

    def count_visited(self):
        return np.count_nonzero(self.history.any(axis=2))


def distinct_positions(grid_txt):
    grid = Grid(grid_txt)
    pos = np.array(grid.find_guard_position())
    direction = get_direction(grid.get(pos))
    out_of_bounds = False

    while not out_of_bounds:
        grid.mark_visited(pos, direction)
        pos, direction, out_of_bounds = grid.next_state(pos, direction)

    return grid.count_visited()


grid_txt = open("inputs/6.txt").read()
distinct_positions(grid_txt)  # 5269

5269

#### 6.2

In [68]:
def looping_obstacles(grid_txt):
    n_looping_obstacles = 0
    grid = Grid(grid_txt)
    initial_pos = np.array(grid.find_guard_position())
    initial_direction = get_direction(grid.get(initial_pos))

    for i in range(grid.n):
        for j in range(grid.m):
            if grid.grid[i, j] != ".":
                continue

            grid.reset_history()
            grid.grid[i, j] = "#"
            pos = initial_pos
            direction = initial_direction
            out_of_bounds = False
            while not out_of_bounds:
                if grid.already_visited(pos, direction):
                    n_looping_obstacles += 1
                    break
                grid.mark_visited(pos, direction)
                pos, direction, out_of_bounds = grid.next_state(pos, direction)

            grid.grid[i, j] = "."

    return n_looping_obstacles


looping_obstacles(grid_txt)  # 1957


1957

#### 7.1

In [69]:
def get_equations(equations_txt):
    return [
        (int(test), np.fromstring(numbers, int, sep=" "))
        for test, numbers in (equation.split(": ") for equation in equations_txt.splitlines())
    ]

In [70]:
def check_combinations(test, numbers, current_value=0):
    if numbers.size == 0:
        return test == current_value
    if current_value > test:
        return False
    return check_combinations(test, numbers[1:], current_value=current_value + numbers[0]) or check_combinations(
        test, numbers[1:], current_value=current_value * numbers[0]
    )


equations_txt = open("inputs/7.txt").read()
equations = get_equations(equations_txt)

sum(test for test, numbers in equations if check_combinations(test, numbers))

20281182715321

#### 7.2

In [71]:
def check_combinations_with_concat(test, numbers, current_value=0):
    if numbers.size == 0:
        return test == current_value
    if current_value > test:
        return False
    return (
        check_combinations_with_concat(test, numbers[1:], current_value=current_value + numbers[0])
        or check_combinations_with_concat(test, numbers[1:], current_value=current_value * numbers[0])
        or check_combinations_with_concat(test, numbers[1:], current_value=int(str(current_value) + str(numbers[0])))
    )

In [72]:
sum(test for test, numbers in equations if check_combinations_with_concat(test, numbers))

159490400628354

#### 8.1

In [73]:
def get_antenna_locs(antenna_map):
    locs = defaultdict(list)
    for i, line in enumerate(antenna_map):
        for j, symbol in enumerate(line):
            if symbol != ".":
                locs[symbol].append(np.array([i, j]))
    return locs


antenna_map_txt = open("inputs/8.txt").read()

antenna_map_example_txt = """\
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............"""

In [74]:
def within_bounds(coords, max_coords):
    return (coords >= 0).all() and (coords < max_coords).all()


def get_n_antinodes(antenna_map_txt):
    antenna_map = antenna_map_txt.splitlines()
    antenna_locs = get_antenna_locs(antenna_map)

    antinode_locs = set()
    max_coords = np.array([len(antenna_map), len(antenna_map[0])])
    for freq_locs in antenna_locs.values():
        for c1, c2 in combinations(freq_locs, 2):
            diff = c2 - c1
            for antinode in (c1 - diff, c2 + diff):
                if within_bounds(antinode, max_coords):
                    antinode_locs.add(tuple(antinode))

    return len(antinode_locs)


get_n_antinodes(antenna_map_txt)

344

#### 8.2

In [75]:
def get_n_resonant_antinodes(antenna_map_txt):
    antenna_map = antenna_map_txt.splitlines()
    antenna_locs = get_antenna_locs(antenna_map)

    antinode_locs = set()
    max_coords = np.array([len(antenna_map), len(antenna_map[0])])

    for freq_locs in antenna_locs.values():
        for c1, c2 in combinations(freq_locs, 2):
            for pos, offset in ((c1.copy(), c1 - c2), (c2.copy(), c2 - c1)):
                while within_bounds(pos, max_coords):
                    antinode_locs.add(tuple(pos))
                    pos += offset
    return len(antinode_locs)


get_n_resonant_antinodes(antenna_map_txt)

1182

#### 9.1

In [76]:
disk_map_txt = open("inputs/9.txt").read()[:-1]


def position_sum(curr_pos, new_pos):
    return (new_pos + curr_pos - 1) * (new_pos - curr_pos) // 2


def update_checksum_and_position(checksum, curr_pos, digit, digit_idx):
    new_pos = curr_pos + digit
    checksum += digit_idx * position_sum(curr_pos, new_pos)
    return checksum, new_pos


def reordered_checksum(disk_map_txt):
    digits = list(map(int, disk_map_txt[::2]))
    spaces = list(map(int, disk_map_txt[1::2]))
    checksum = 0
    curr_pos = 0
    end_idx = len(digits) - 1
    for idx, _ in enumerate(digits):
        checksum, curr_pos = update_checksum_and_position(checksum, curr_pos, digits[idx], idx)

        if end_idx <= idx:
            break

        while end_idx > idx and spaces[idx] > 0:
            filled_positions = min(digits[end_idx], spaces[idx])
            spaces[idx] -= filled_positions
            digits[end_idx] -= filled_positions
            checksum, curr_pos = update_checksum_and_position(checksum, curr_pos, filled_positions, end_idx)
            if digits[end_idx] == 0:
                end_idx -= 1

    return checksum


disk_map_example_txt = "2333133121414131402"
reordered_checksum(disk_map_example_txt)

1928

#### 9.2

In [77]:
def defragmented_checksum(disk_map_txt):
    digits_per_pos = [[(dig_id, int(symbol))] for dig_id, symbol in enumerate(disk_map_txt[::2])]
    spaces = list(map(int, disk_map_txt[1::2])) + [0]

    for end_idx, _ in reversed(list(enumerate(digits_per_pos))):
        dig_id, digit = digits_per_pos[end_idx][0]
        for space_idx, _ in enumerate(spaces):
            if space_idx >= end_idx:
                break
            if spaces[space_idx] >= digit:
                spaces[space_idx] -= digit
                digits_per_pos[space_idx].append((dig_id, digit))
                spaces[dig_id - 1] += digit
                digits_per_pos[dig_id][0] = (dig_id, 0)
                break

    curr_pos = 0
    checksum = 0
    for idx, digit_list in enumerate(digits_per_pos):
        for dig_id, digit in digit_list:
            checksum, curr_pos = update_checksum_and_position(checksum, curr_pos, digit, dig_id)
        curr_pos += spaces[idx]

    return checksum


defragmented_checksum(disk_map_txt)

6408966547049

#### 10.1

In [78]:
topo_map_txt = open("inputs/10.txt").read()

In [79]:
def map_score(topo_map_txt):
    topo_map = np.array([list(map(int, line)) for line in topo_map_txt.splitlines()])

    def out_of_bounds(i, j):
        return i < 0 or j < 0 or i >= topo_map.shape[0] or j >= topo_map.shape[1]

    deltas = np.array(
        [
            [0, 1],
            [1, 0],
            [0, -1],
            [-1, 0],
        ]
    )
    score = 0
    trailheads = np.argwhere(topo_map == 0)
    for trailhead in trailheads:
        visited = np.zeros_like(topo_map, dtype=bool)
        stack = [trailhead]
        while stack:
            top = stack.pop()
            if visited[tuple(top)]:
                continue

            if topo_map[tuple(top)] == 9:
                score += 1
            visited[tuple(top)] = True
            for delta in deltas:
                new_pos = top + delta
                if (
                    not out_of_bounds(*new_pos)
                    and not visited[tuple(new_pos)]
                    and topo_map[tuple(new_pos)] == topo_map[tuple(top)] + 1
                ):
                    stack.append(new_pos)
    return score


map_score(topo_map_txt)

566

#### 10.2

In [80]:
def map_score(topo_map_txt):
    topo_map = np.array([list(map(int, line)) for line in topo_map_txt.splitlines()])

    def out_of_bounds(i, j):
        return i < 0 or j < 0 or i >= topo_map.shape[0] or j >= topo_map.shape[1]

    deltas = np.array(
        [
            [0, 1],
            [1, 0],
            [0, -1],
            [-1, 0],
        ]
    )
    score = 0
    trailheads = np.argwhere(topo_map == 0)

    visited = np.zeros_like(topo_map, dtype=int)
    for pos in trailheads:
        visited[tuple(pos)] = 1
    new_stack = list(trailheads)
    height = 0

    while new_stack:
        stack = new_stack
        new_stack = []
        for pos in stack:
            for delta in deltas:
                new_pos = pos + delta
                if not out_of_bounds(*new_pos) and topo_map[tuple(new_pos)] == height + 1:
                    if not visited[tuple(new_pos)]:
                        new_stack.append(new_pos)
                    visited[tuple(new_pos)] += visited[tuple(pos)]
        height += 1

    score = sum(visited[tuple(pos)] for pos in np.argwhere(topo_map == 9))
    return score


map_score(topo_map_txt)

np.int64(1324)

#### 11.1

In [81]:
def evolve(number):
    if number == 0:
        return [1]

    digits = str(number)
    n_digits = len(digits)
    if n_digits % 2 == 0:
        return [int(digits[: n_digits // 2]), int(digits[n_digits // 2 :])]

    return [number * 2024]


def get_n_stones(stones_txt, n_blinks=25):
    stone_counts = dict(Counter(map(int, stones_txt.split())))

    for i in range(n_blinks):
        next_counts = defaultdict(int)
        for number, count in stone_counts.items():
            for evolved_number in evolve(number):
                next_counts[evolved_number] += count
        stone_counts = next_counts
    return sum(stone_counts.values())


stones_txt = open("inputs/11.txt").read()

get_n_stones(stones_txt, n_blinks=25)

190865

#### 11.2

In [82]:
get_n_stones(stones_txt, n_blinks=75)

225404711855335

#### 12.1

In [83]:
def add_delta(coords, delta):
    return (coords[0] + delta[0], coords[1] + delta[1])


def fencing_price(garden_txt):
    garden = np.array(list(map(list, garden_txt.split())))

    visited = np.zeros_like(garden, dtype=bool)
    deltas = (
        (-1, 0),
        (0, 1),
        (1, 0),
        (0, -1),
    )

    def is_connected(nb_coords, plant_type):
        return not out_of_bounds(*nb_coords) and garden[nb_coords] == plant_type

    def out_of_bounds(i, j):
        return i < 0 or j < 0 or i >= garden.shape[0] or j >= garden.shape[1]

    def dfs(coords, plant_type, component):
        component.append(coords)
        visited[coords] = True
        perimeter = 4
        for delta in deltas:
            nb_coords = add_delta(coords, delta)
            if is_connected(nb_coords, plant_type):
                perimeter -= 1
                if not visited[nb_coords]:
                    perimeter += dfs(nb_coords, plant_type, component)
        return perimeter

    price = 0
    for coords, curr_plot in np.ndenumerate(garden):
        if not visited[coords]:
            connected_component = []
            perimeter = dfs(coords, curr_plot, connected_component)
            price += len(connected_component) * perimeter
    return price

In [84]:
garden_txt = open("inputs/12.txt").read()

fencing_price(garden_txt)

1464678

#### 12.2

In [85]:
def discounted_fencing_price(garden_txt):
    garden = np.array(list(map(list, garden_txt.split())))

    visited = np.zeros_like(garden, dtype=bool)
    deltas = (
        (-1, 0),
        (0, 1),
        (1, 0),
        (0, -1),
    )

    def is_connected(nb_coords, plant_type):
        return not out_of_bounds(*nb_coords) and garden[nb_coords] == plant_type

    def out_of_bounds(i, j):
        return i < 0 or j < 0 or i >= garden.shape[0] or j >= garden.shape[1]

    def dfs(coords, plant_type, component):
        component.append(coords)
        visited[coords] = True

        for delta in deltas:
            nb_coords = add_delta(coords, delta)
            if is_connected(nb_coords, plant_type):
                if not visited[nb_coords]:
                    dfs(nb_coords, plant_type, component)

    price = 0
    for coords, curr_plot in np.ndenumerate(garden):
        if not visited[coords]:
            connected_component = []
            dfs(coords, curr_plot, connected_component)
            sides = 0
            for i, coords in enumerate(connected_component):
                for direction in Direction:
                    delta = deltas[direction.value]
                    if not is_connected(add_delta(coords, delta), curr_plot):
                        left_rotated_nb_coords = add_delta(coords, deltas[rotate_left(direction).value])
                        if not is_connected(left_rotated_nb_coords, curr_plot) or is_connected(
                            add_delta(left_rotated_nb_coords, delta), curr_plot
                        ):
                            sides += 1

            price += len(connected_component) * sides
    return price


discounted_fencing_price(garden_txt)

877492

#### 13.1

In [86]:
def process_machine_input(line):
    return re.findall(r"Button A: X\+([0-9]+), Y\+([0-9]+)\nButton B: X\+([0-9]+), Y\+([0-9]+)\nPrize: X=([0-9]+), Y=([0-9]+)", line)[
        0
    ]


def min_tokens(machine_input_line, fix_unit_conversion=False):
    x_a, y_a, x_b, y_b, price_x, price_y = map(int, process_machine_input(machine_input_line))
    A = np.array(
        [
            [x_a, x_b],
            [y_a, y_b],
        ]
    )

    assert np.linalg.det(A) != 0

    p = np.array([price_x, price_y])
    if fix_unit_conversion:
        p += 10000000000000

    n_tokens = np.linalg.solve(A, p).round().astype(int)

    if not np.array_equal(A @ n_tokens, p):
        return 0

    return 3 * n_tokens[0] + n_tokens[1]


machine_inputs_txt = open("inputs/13.txt").read()[:-1].split("\n\n")
sum(map(min_tokens, machine_inputs_txt))


np.int64(37686)

#### 13.2

In [87]:
sum(min_tokens(line, fix_unit_conversion=True) for line in machine_inputs_txt)

np.int64(77204516023437)

#### 14.1

In [88]:
robot_inputs_txt = open("inputs/14.txt").read()

In [89]:
def safety_factor(robot_inputs_txt, bathroom_shape, n_seconds=100):
    bathroom_shape = np.array(bathroom_shape)
    positions, velocities = np.split(
        np.array(re.findall(r"p=([0-9]+),([0-9]+) v=(-?[0-9]+),(-?[0-9]+)", robot_inputs_txt), dtype=int),
        2,
        axis=1,
    )

    new_positions = (positions + velocities * n_seconds) % bathroom_shape
    split_lines = bathroom_shape // 2

    ans = 1
    for in_quadrant in [
        (new_positions[:, 0] < split_lines[0]) & (new_positions[:, 1] < split_lines[1]),
        (new_positions[:, 0] < split_lines[0]) & (new_positions[:, 1] > split_lines[1]),
        (new_positions[:, 0] > split_lines[0]) & (new_positions[:, 1] < split_lines[1]),
        (new_positions[:, 0] > split_lines[0]) & (new_positions[:, 1] > split_lines[1]),
    ]:
        ans *= np.count_nonzero(in_quadrant)
    return ans


safety_factor(robot_inputs_txt, (101, 103))

209409792

In [90]:
def largest_component(positions, bathroom_shape):
    grid = np.zeros(bathroom_shape, int)
    np.add.at(grid, (positions[:, 0], positions[:, 1]), 1)

    assert np.sum(grid) == len(positions)
    visited = np.zeros(grid.shape, bool)

    def dfs(coords, n_robots):
        if visited[coords]:
            return n_robots
        visited[coords] = True
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue

                x_next = coords[0] + dx
                y_next = coords[1] + dy
                if x_next < 0 or y_next < 0 or x_next >= bathroom_shape[0] or y_next >= bathroom_shape[1]:
                    continue
                if grid[x_next, y_next] > 0:
                    n_robots = dfs((x_next, y_next), n_robots)
        return n_robots + 1

    return max(dfs((x, y), 0) for x, y in positions)


def christmas_tree_time(robot_inputs_txt, bathroom_shape, t_max=100, verbose=False):
    bathroom_shape = np.array(bathroom_shape)
    positions, velocities = np.split(
        np.array(re.findall(r"p=([0-9]+),([0-9]+) v=(-?[0-9]+),(-?[0-9]+)", robot_inputs_txt), dtype=int),
        2,
        axis=1,
    )

    best_so_far = 1
    for t in range(1, t_max + 1):
        positions = (positions + velocities) % bathroom_shape
        conn = largest_component(positions, bathroom_shape)
        if conn > best_so_far:
            if conn > positions.shape[0] * 0.1:
                if verbose:
                    grid = np.zeros(bathroom_shape, int)
                    np.add.at(grid, (positions[:, 0], positions[:, 1]), 1)
                    for line in grid.T:
                        print("".join([str(val) if val > 0 else " " for val in line]))
                return t
            best_so_far = conn
    return -1


christmas_tree_time(robot_inputs_txt, (101, 103), t_max=100000)

8006

#### 15.1

In [91]:
warehouse_txt = open("inputs/15.txt").read()[:-1]


In [92]:
def move_boxes(grid, pos, dx):
    new_pos = pos + dx
    while (new_pos >= 0).all() and (new_pos < grid.shape).all() and grid[tuple(new_pos)] != "#":
        if grid[tuple(new_pos)] == ".":
            return new_pos

        new_pos += dx
    return None


def simulate_warehouse(grid, dx_vecs):
    robot_pos = np.argwhere(grid == "@")[0]
    for dx in dx_vecs:
        new_pos = robot_pos + dx
        if grid[tuple(new_pos)] == "#":
            continue
        if grid[tuple(new_pos)] == "O":
            new_box_position = move_boxes(grid, new_pos, dx)
            if new_box_position is None:
                continue
            grid[tuple(new_box_position)] = "O"
        grid[tuple(new_pos)] = "@"
        grid[tuple(robot_pos)] = "."
        robot_pos = new_pos


def gps_sum(warehouse_txt):
    grid_txt, instructions_txt = warehouse_txt.split("\n\n")
    instructions_txt = instructions_txt.replace("\n", "")
    grid = np.array(list(map(list, grid_txt.split())))
    dx_vecs = [get_dx(get_direction(instruction)) for instruction in instructions_txt]
    simulate_warehouse(grid, dx_vecs)

    box_coords = np.argwhere(grid == "O")
    return 100 * np.sum(box_coords[:, 0]) + np.sum(box_coords[:, 1])


gps_sum(warehouse_txt)

np.int64(1426855)

#### 15.2

In [93]:
def move_scaled_boxes(grid, pos, direction, box_coords, visited=None):
    if visited is None:
        visited = set()
    box_positions = (pos, pos + get_dx(Direction.RIGHT)) if grid[tuple(pos)] == "[" else (pos + get_dx(Direction.LEFT), pos)
    if tuple(box_positions[0]) in visited:
        return True
    visited.add(tuple(box_positions[0]))
    dx = get_dx(direction)

    new_left_pos = box_positions[0] + dx
    new_right_pos = box_positions[1] + dx

    if grid[tuple(new_left_pos)] == "#" or grid[tuple(new_right_pos)] == "#":
        return False

    if direction in [Direction.UP, Direction.DOWN]:
        if grid[tuple(new_left_pos)] == "[":
            if not move_scaled_boxes(grid, new_left_pos, direction, box_coords, visited=visited):
                return False
        else:
            if grid[tuple(new_left_pos)] == "]":
                if not move_scaled_boxes(grid, new_left_pos, direction, box_coords, visited=visited):
                    return False

            if grid[tuple(new_right_pos)] == "[":
                if not move_scaled_boxes(grid, new_right_pos, direction, box_coords, visited=visited):
                    return False
    else:
        new_pos = new_left_pos if direction == Direction.LEFT else new_right_pos
        if grid[tuple(new_pos)] != "." and not move_scaled_boxes(grid, new_pos, direction, box_coords, visited=visited):
            return False

    box_coords.append(box_positions)
    return True


def simulate_scaled_warehouse(grid, directions):
    robot_pos = np.argwhere(grid == "@")[0]
    for direction in directions:
        dx = get_dx(direction)
        new_pos = robot_pos + dx
        if grid[tuple(new_pos)] == "#":
            continue
        if grid[tuple(new_pos)] in "[]":
            original_boxes_coords = []
            if not move_scaled_boxes(grid, new_pos, direction, original_boxes_coords):
                continue
            assert len(original_boxes_coords) > 0
            for left_box_coords, right_box_coords in original_boxes_coords:
                grid[tuple(left_box_coords)] = "."
                grid[tuple(right_box_coords)] = "."
                grid[tuple(left_box_coords + dx)] = "["
                grid[tuple(right_box_coords + dx)] = "]"

        grid[tuple(new_pos)] = "@"
        grid[tuple(robot_pos)] = "."
        robot_pos = new_pos


def scaled_gps_sum(warehouse_txt):
    grid_txt, instructions_txt = warehouse_txt.split("\n\n")
    grid_txt = grid_txt.replace("#", "##").replace(".", "..").replace("O", "[]").replace("@", "@.")
    instructions_txt = instructions_txt.replace("\n", "")
    grid = np.array(list(map(list, grid_txt.split())))
    directions = [get_direction(instruction) for instruction in instructions_txt]
    simulate_scaled_warehouse(grid, directions)

    box_coords = np.argwhere(grid == "[")
    return 100 * np.sum(box_coords[:, 0]) + np.sum(box_coords[:, 1])


scaled_gps_sum(warehouse_txt)

np.int64(1404917)

#### 16.1

In [94]:
maze_txt = open("inputs/16.txt").read()[:-1]

In [95]:
@dataclass(order=True, frozen=True)
class MazeState:
    dist: int
    coords: np.array = field(compare=False)
    direction: Direction = field(compare=False)

    def next_states(self):
        left_direction = rotate_left(self.direction)
        right_direction = rotate_right(self.direction)
        return (
            MazeState(self.dist + 1, self.coords + get_dx(self.direction), self.direction),
            MazeState(self.dist + 1001, self.coords + get_dx(left_direction), left_direction),
            MazeState(self.dist + 1001, self.coords + get_dx(right_direction), right_direction),
        )

    def get_idx_tuple(self):
        return (*self.coords, self.direction.value)


def best_maze_score(maze_txt):
    grid = np.array(list(map(list, maze_txt.split())))
    start_state = MazeState(0, np.argwhere(grid == "S")[0], Direction.RIGHT)
    end_coords = np.argwhere(grid == "E")[0]

    dist = np.full((*grid.shape, 4), np.iinfo(int).max, dtype=int)
    queue = [start_state]
    while queue:
        state = heappop(queue)
        if (state.coords == end_coords).all():
            return state.dist

        for next_state in state.next_states():
            next_idx_tuple = next_state.get_idx_tuple()
            if grid[tuple(next_state.coords)] != "#" and next_state.dist < dist[next_idx_tuple]:
                dist[next_idx_tuple] = next_state.dist
                heappush(queue, next_state)
    return -1


best_maze_score(maze_txt)

123540

#### 16.2

In [96]:
def shortest_path_tiles(maze_txt):
    grid = np.array(list(map(list, maze_txt.split())))
    start_state = MazeState(0, np.argwhere(grid == "S")[0], Direction.RIGHT)
    end_coords = np.argwhere(grid == "E")[0]
    max_dist = np.iinfo(int).max
    dist = np.full((*grid.shape, 4), max_dist, dtype=int)
    prev = defaultdict(list)
    queue = [start_state]
    while queue:
        state = heappop(queue)
        if state.dist > max_dist:
            break
        if (state.coords == end_coords).all():
            max_dist = state.dist

        idx_tuple = state.get_idx_tuple()

        for next_state in state.next_states():
            next_idx_tuple = next_state.get_idx_tuple()
            if grid[tuple(next_state.coords)] != "#":
                if next_state.dist == dist[next_idx_tuple]:
                    prev[next_idx_tuple].append(idx_tuple)
                elif next_state.dist < dist[next_idx_tuple]:
                    prev[next_idx_tuple] = [idx_tuple]
                    dist[next_idx_tuple] = next_state.dist
                    heappush(queue, next_state)

    visited = np.zeros_like(dist, dtype=bool)
    stack = []
    for direction in Direction:
        idx_tuple = (*end_coords, direction.value)
        if idx_tuple in prev:
            stack.append(idx_tuple)

    while stack:
        idx_tuple = stack.pop()
        if visited[idx_tuple]:
            continue
        visited[idx_tuple] = True
        for prev_tuple in prev[idx_tuple]:
            stack.append(prev_tuple)

    return np.count_nonzero(visited.any(axis=-1))


shortest_path_tiles(maze_txt)

665

#### 17.1

In [97]:
program_txt = open("inputs/17.txt").read()[:-1]

In [98]:
def run_program(program_txt):
    register_txt, opcodes_txt = program_txt.split("\n\n")
    registers = [int(line.split(": ")[1]) for line in register_txt.splitlines()]
    program = list(map(int, opcodes_txt.split(": ")[1].split(",")))
    outputs = []

    i = 0
    while i < len(program) - 1:
        opcode, operand = program[i : i + 2]
        combo = operand if operand < 4 or operand > 6 else registers[operand - 4]
        match opcode:
            case 0:
                registers[0] //= 2**combo
            case 1:
                registers[1] ^= operand
            case 2:
                registers[1] = combo & 7
            case 3:
                if registers[0] != 0:
                    i = operand - 2
            case 4:
                registers[1] ^= registers[2]
            case 5:
                outputs.append(combo & 7)
            case 6:
                registers[1] = registers[0] // (2**combo)
            case 7:
                registers[2] = registers[0] // (2**combo)
        i += 2
    return ",".join(map(str, outputs))


run_program(program_txt)


'2,3,4,7,5,7,3,0,7'

#### 17.2

In [99]:
def last_output_digit(A):
    return (A & 7) ^ 5 ^ ((A >> ((A & 7) ^ 2)) & 7)


def run_test_program(A_val):
    ans = []
    while A_val > 0:
        ans.append(str(last_output_digit(A_val)))
        A_val //= 8
    return ",".join(ans)


In [100]:
def find_A_value():
    opcodes_txt = program_txt.split("Program: ")[1]
    program = list(map(int, opcodes_txt.split(",")))

    def backtrack(idx, curr_A=0):
        curr_A *= 8
        for i in range(8):
            next_A = curr_A + i
            if last_output_digit(next_A) == program[-idx - 1]:
                if idx == len(program) - 1:
                    return next_A
                final_A = backtrack(idx + 1, curr_A=next_A)
                if final_A is not None:
                    return final_A
        return None

    ans = backtrack(0)
    assert run_test_program(ans) == opcodes_txt

    return ans


find_A_value()

190384609508367

#### 18.1

In [101]:
falling_bytes_txt = open("inputs/18.txt").read()[:-1]

In [102]:
def memory_grid_steps(falling_bytes_txt, grid_size=71, n_bytes=1024):
    byte_grid = np.zeros((grid_size, grid_size), dtype=bool)
    falling_bytes = np.roll(
        np.genfromtxt(falling_bytes_txt.splitlines(), delimiter=",", dtype=int)[:n_bytes],
        axis=1,
        shift=1,
    )
    byte_grid[falling_bytes[:, 0], falling_bytes[:, 1]] = 1

    visited = np.zeros_like(byte_grid)
    start_pos = np.array([0, 0])
    end_pos = np.array([grid_size - 1, grid_size - 1])
    distance = 0
    next_queue = [start_pos]

    while next_queue:
        queue = next_queue
        next_queue = []
        distance += 1

        for curr_pos in queue:
            for direction in Direction:
                next_pos = curr_pos + get_dx(direction)
                if (
                    (next_pos < 0).any()
                    or (next_pos >= byte_grid.shape).any()
                    or visited[tuple(next_pos)]
                    or byte_grid[tuple(next_pos)]
                ):
                    continue
                if (next_pos == end_pos).all():
                    return distance
                visited[tuple(next_pos)] = True
                next_queue.append(next_pos)

    return -1


memory_grid_steps(falling_bytes_txt)

246

#### 18.2

In [103]:
def blocking_byte(falling_bytes_txt):
    falling_bytes_lines = falling_bytes_txt.splitlines()
    low = 1024
    high = len(falling_bytes_lines) - 1

    while low < high:
        mid = (high + low) // 2
        if memory_grid_steps(falling_bytes_txt, n_bytes=mid) != -1:
            low = mid + 1
        else:
            high = mid

    return falling_bytes_lines[low - 1]


blocking_byte(falling_bytes_txt)

'22,50'

#### 19.1

In [46]:
towels_input_txt = open("inputs/19.txt").read()[:-1]

In [50]:
class PatternNode:
    def __init__(self, pattern=None):
        self.children = dict()
        self.is_leaf = False
        if pattern is None:
            self.stripe = ""
        else:
            self.stripe = pattern[0]
            self.addChild(pattern[1:])

    def addChild(self, pattern):
        if pattern:
            if pattern[0] not in self.children:
                self.children[pattern[0]] = PatternNode(pattern)
            else:
                self.children[pattern[0]].addChild(pattern[1:])
        else:
            self.is_leaf = True


def is_possible(design, rootNode):
    n = len(design)
    can_be_reached = np.zeros(n + 1, dtype=bool)
    can_be_reached[0] = True
    for i in range(n):
        if not can_be_reached[i]:
            continue
        currNode = rootNode
        for j in range(i, n):
            if design[j] not in currNode.children:
                break
            currNode = currNode.children[design[j]]
            if currNode.is_leaf:
                can_be_reached[j + 1] = True
    return can_be_reached[-1]


def possible_designs(towels_input_txt):
    towel_patterns_txt, designs_txt = towels_input_txt.split("\n\n")
    towel_patterns = towel_patterns_txt.split(", ")
    designs = designs_txt.split("\n")

    root = PatternNode()
    for pattern in towel_patterns:
        root.addChild(pattern)

    return sum(is_possible(design, root) for design in designs)


possible_designs(towels_input_txt)

np.int64(267)

#### 19.2

In [51]:
def n_design_ways(design, rootNode):
    n = len(design)
    n_ways = np.zeros(n + 1, dtype=int)
    n_ways[0] = 1
    for i in range(n):
        if n_ways[i] == 0:
            continue
        currNode = rootNode
        for j in range(i, n):
            if design[j] not in currNode.children:
                break
            currNode = currNode.children[design[j]]
            if currNode.is_leaf:
                n_ways[j + 1] += n_ways[i]
    return n_ways[-1]


def possible_design_ways(towels_input_txt):
    towel_patterns_txt, designs_txt = towels_input_txt.split("\n\n")
    towel_patterns = towel_patterns_txt.split(", ")
    designs = designs_txt.split("\n")

    root = PatternNode()
    for pattern in towel_patterns:
        root.addChild(pattern)

    return sum(n_design_ways(design, root) for design in designs)


possible_design_ways(towels_input_txt)

np.int64(796449099271652)

#### 20.1

In [79]:
racetrack_txt = open("inputs/20.txt").read()[:-1]

In [80]:
def find_race_cheats(racetrack_txt):
    grid = np.array(list(map(list, racetrack_txt.split())))
    max_dist = np.iinfo(int).max
    dist = np.full(grid.shape, max_dist, dtype=int)

    start_pos = np.argwhere(grid == "S")[0]
    dist[tuple(start_pos)] = 0
    end_pos = np.argwhere(grid == "E")[0]
    curr_pos = start_pos
    while (curr_pos != end_pos).any():
        for direction in Direction:
            next_pos = curr_pos + get_dx(direction)
            if grid[tuple(next_pos)] != "#" and dist[tuple(next_pos)] == max_dist:
                dist[tuple(next_pos)] = dist[tuple(curr_pos)] + 1
                curr_pos = next_pos

    saving_counts = defaultdict(int)
    for i in range(1, grid.shape[0] - 1):
        for j in range(1, grid.shape[1] - 1):
            pos = np.array([i, j])
            nearby_dists = [curr_dist for direction in Direction if (curr_dist := dist[tuple(pos + get_dx(direction))]) != max_dist]
            for dist1, dist2 in combinations(sorted(nearby_dists), 2):
                saving_counts[dist2 - dist1 - 2] += 1

    return sum(count for saving, count in saving_counts.items() if saving >= 100)


find_race_cheats(racetrack_txt)

1438

#### 20.2

In [ ]:
def find_longer_race_cheats(racetrack_txt, n_picoseconds=20):
    grid = np.array(list(map(list, racetrack_txt.split())))
    max_dist = np.iinfo(int).max
    dist = np.full(grid.shape, max_dist, dtype=int)

    start_pos = np.argwhere(grid == "S")[0]
    dist[tuple(start_pos)] = 0
    end_pos = np.argwhere(grid == "E")[0]
    curr_pos = start_pos
    while (curr_pos != end_pos).any():
        for direction in Direction:
            next_pos = curr_pos + get_dx(direction)
            if grid[tuple(next_pos)] != "#" and dist[tuple(next_pos)] == max_dist:
                dist[tuple(next_pos)] = dist[tuple(curr_pos)] + 1
                curr_pos = next_pos

    saving_counts = defaultdict(int)
    for i in range(1, grid.shape[0] - 1):
        for j in range(1, grid.shape[1] - 1):
            curr_dist = dist[i, j]
            if curr_dist == max_dist:
                continue
            for d_i in range(max(-n_picoseconds, 1 - i), min(n_picoseconds + 1, grid.shape[0] - 1 - i)):
                for d_j in range(max(-n_picoseconds + abs(d_i), 1 - j), min(n_picoseconds - abs(d_i) + 1, grid.shape[1] - 1 - j)):
                    next_dist = dist[i + d_i, j + d_j]
                    if next_dist != max_dist:
                        saving_counts[next_dist - curr_dist - abs(d_i) - abs(d_j)] += 1
    return sum(count for saving, count in saving_counts.items() if saving >= 100)


find_longer_race_cheats(racetrack_txt, n_picoseconds=20)

1026446

#### 21.1

In [6]:
example_door_codes = """\
029A
980A
179A
456A
379A""".split("\n")
door_codes = open("inputs/21.txt").read().split()

In [64]:
def directions(single_diff):
    if single_diff[0] < 0:
        return "^" * abs(single_diff[0])
    elif single_diff[0] > 0:
        return "v" * single_diff[0]
    elif single_diff[1] < 0:
        return "<" * abs(single_diff[1])
    return ">" * single_diff[1]


numeric_positions = {
    "0": np.array([3, 1]),
    "A": np.array([3, 2]),
    **{str(i): np.array([2 - ((i - 1) // 3), (i - 1) % 3]) for i in range(1, 10)},
}


directional_positions = {
    "<": np.array([1, 0]),
    "^": np.array([0, 1]),
    ">": np.array([1, 2]),
    "v": np.array([1, 1]),
    "A": np.array([0, 2]),
}


def get_route_list(start_pos, end_pos, gap_pos):
    diff = end_pos - start_pos
    route_list = []
    if np.count_nonzero(diff) == 0:
        route_list.append("A")
    elif np.count_nonzero(diff) == 1:
        route_list.append(directions(end_pos - start_pos) + "A")
    else:
        for p1, p2 in ((start_pos, end_pos), (end_pos, start_pos)):
            midway_pos = np.array([p1[0], p2[1]])
            if (midway_pos == gap_pos).all():
                continue
            route_list.append(directions(midway_pos - start_pos) + directions(end_pos - midway_pos) + "A")
    return route_list


def possible_numeric_routes(curr_val, next_val):
    return get_route_list(numeric_positions[curr_val], numeric_positions[next_val], np.array([3, 0]))


def possible_directional_routes(curr_val, next_val):
    return get_route_list(directional_positions[curr_val], directional_positions[next_val], np.array([0, 0]))


def complexity_sum(door_codes, n_robots):
    directional_keys = "<^>vA"
    dp = np.zeros((n_robots + 1, len(directional_keys), len(directional_keys)), dtype=np.uint64)
    dp[0] = 1

    def shortest_robot_seq_length(sequence, depth):
        key_indices = np.array([directional_keys.index(key) for key in ("A" + sequence)])
        return np.sum(dp[depth - 1, key_indices[:-1], key_indices[1:]])

    for r in range(1, n_robots + 1):
        for i1, key1 in enumerate(directional_keys):
            for i2, key2 in enumerate(directional_keys):
                dp[r, i1, i2] = min(shortest_robot_seq_length(route, r) for route in possible_directional_routes(key1, key2))

    complexity = 0
    for door_code in door_codes:
        length = 0
        for curr_val, next_val in zip("A" + door_code, door_code):
            route_list = possible_numeric_routes(curr_val, next_val)
            length += min(shortest_robot_seq_length(route, depth=n_robots) for route in route_list)
        complexity += int(door_code[:-1]) * length
    return complexity


complexity_sum(door_codes, 3)

np.uint64(219366)

#### 21.2

In [65]:
complexity_sum(door_codes, n_robots=26)

np.uint64(271631192020464)

#### 22.1

In [63]:
secret_numbers_txt = open("inputs/22.txt").read()

In [64]:
def evolve_number(number):
    number = (number ^ (number << 6)) % (2**24)
    number = (number ^ (number >> 5)) % (2**24)
    number = (number ^ (number << 11)) % (2**24)
    return number


def secret_number_sum(initial_numbers_txt):
    sum = 0
    initial_numbers = list(map(int, initial_numbers_txt.split()))
    for initial_number in initial_numbers:
        number = initial_number
        for _ in range(2000):
            number = evolve_number(number)
        sum += number

    return sum


secret_number_sum(secret_numbers_txt)

14476723788

#### 22.2

In [ ]:
def max_bananas(initial_numbers_txt):
    initial_numbers = list(map(int, initial_numbers_txt.split()))
    total_bananas_per_change = np.zeros((19, 19, 19, 19), dtype=int)
    for initial_number in initial_numbers:
        seq_found = np.zeros(total_bananas_per_change.shape, dtype=bool)
        numbers = np.empty(2001, dtype=int)
        numbers[0] = initial_number
        for i in range(1, 2001):
            numbers[i] = evolve_number(numbers[i - 1])

        numbers = numbers % 10
        diffs = np.diff(numbers)
        for i in range(4, 2001):
            idcs = tuple(diffs[i - 4 : i])
            if not seq_found[idcs]:
                seq_found[idcs] = True
                total_bananas_per_change[idcs] += numbers[i]
    return total_bananas_per_change.max()


max_bananas(secret_numbers_txt)


np.int64(1630)

#### 23.1

In [145]:
network_txt = open("inputs/23.txt").read()

In [146]:
def find_lan_triplets(network_txt):
    adj_list = defaultdict(set)
    for line in network_txt.split():
        c1, c2 = line.split("-")
        adj_list[c1].add(c2)
        adj_list[c2].add(c1)

    n_triplets = 0
    for c1, nbs in adj_list.items():
        if c1[0] != "t":
            continue
        for c2, c3 in combinations(nbs, 2):
            if c3 in adj_list[c2]:
                if (c2[0] != "t" or c1 < c2) and (c3[0] != "t" or c1 < c3):
                    n_triplets += 1

    return n_triplets


find_lan_triplets(network_txt)

1314

#### 23.2

In [175]:
from collections.abc import Hashable


def maximum_clique(adj_list: dict[Hashable, set[Hashable]]):
    maximal_cliques = []

    def bron_kerbosch(R, P, X):
        if not P and not X:
            maximal_cliques.append(R)
        while P:
            v = next(iter(P))
            bron_kerbosch(R | {v}, P & adj_list[v], X & adj_list[v])
            P.remove(v)
            X.add(v)

    bron_kerbosch(set(), set(adj_list), set())
    return max(maximal_cliques, key=len)


def find_lan_party(network_txt):
    adj_list = defaultdict(set)
    for line in network_txt.split():
        c1, c2 = line.split("-")
        adj_list[c1].add(c2)
        adj_list[c2].add(c1)

    return ",".join(sorted(maximum_clique(adj_list)))


find_lan_party(network_txt)

'bg,bu,ce,ga,hw,jw,nf,nt,ox,tj,uu,vk,wp'

#### 24.1

In [4]:
gates_input_txt = open("inputs/24.txt").read()

In [20]:
def simulate_gates(gates_input_txt):
    initial_values_lines, gates_lines = (txt.split("\n") for txt in gates_input_txt.split("\n\n"))
    gates_lines = gates_lines[:-1]
    wire_values = {wire: int(value) for wire, value in map(lambda line: line.split(": "), initial_values_lines)}

    gate_mapping = {
        "AND": lambda left, right: left and right,
        "OR": lambda left, right: left or right,
        "XOR": lambda left, right: left ^ right,
    }
    gate_ops = {
        output: (input1, gate_mapping[gate], input2) for input1, gate, input2, _, output in map(lambda line: line.split(), gates_lines)
    }

    def obtain_gate_value(output):
        if output not in wire_values:
            input1, gate_op, input2 = gate_ops[output]
            val1 = obtain_gate_value(input1)
            val2 = obtain_gate_value(input2)
            wire_values[output] = gate_op(val1, val2)

        return wire_values[output]

    output_wires = sorted(filter(lambda wire: wire[2].isdigit(), gate_ops.keys()))
    return sum(obtain_gate_value(output_wire) << i for i, output_wire in enumerate(output_wires))


simulate_gates(gates_input_txt)

51745744348272

#### 24.2

In [31]:
def find_gate_errors(gates_input_txt):
    initial_values_lines, gates_lines = (txt.split("\n") for txt in gates_input_txt.split("\n\n"))
    gates_lines = gates_lines[:-1]
    wire_values = {wire: int(value) for wire, value in map(lambda line: line.split(": "), initial_values_lines)}

    gate_mapping = {
        "AND": lambda left, right: left and right,
        "OR": lambda left, right: left or right,
        "XOR": lambda left, right: left ^ right,
    }
    gate_ops = {output: (input1, gate, input2) for input1, gate, input2, _, output in map(lambda line: line.split(), gates_lines)}

    for g1, g2 in [("z31", "hkh"), ("z27", "bfq"), ("bng", "fjp"), ("hmt", "z18")]:
        val1 = gate_ops[g1]
        val2 = gate_ops[g2]

        gate_ops[g2] = val1
        gate_ops[g1] = val2

    def obtain_gate_value(output):
        if output not in wire_values:
            input1, gate_op, input2 = gate_ops[output]
            val1 = obtain_gate_value(input1)
            val2 = obtain_gate_value(input2)
            wire_values[output] = gate_mapping[gate_op](val1, val2)
            print(input1, gate_op, input2, output)

        return wire_values[output]

    output_wires = sorted(filter(lambda wire: wire[2].isdigit(), gate_ops.keys()))
    for output_wire in output_wires:
        print(f"====== {output_wire} ======")
        obtain_gate_value(output_wire)


find_gate_errors(gates_input_txt)


====== z00 ======
y00 XOR x00 z00
====== z01 ======
y00 AND x00 rjr
x01 XOR y01 sgt
rjr XOR sgt z01
====== z02 ======
x01 AND y01 hqg
rjr AND sgt cff
hqg OR cff fkm
y02 XOR x02 hvb
fkm XOR hvb z02
====== z03 ======
fkm AND hvb hnv
x02 AND y02 rbm
hnv OR rbm bdp
y03 XOR x03 thv
bdp XOR thv z03
====== z04 ======
x04 XOR y04 stt
y03 AND x03 bfs
thv AND bdp rvq
bfs OR rvq cmh
stt XOR cmh z04
====== z05 ======
stt AND cmh mwj
x04 AND y04 pmq
mwj OR pmq ngj
y05 XOR x05 pqj
ngj XOR pqj z05
====== z06 ======
y05 AND x05 rkt
ngj AND pqj ckj
rkt OR ckj wts
x06 XOR y06 vrh
wts XOR vrh z06
====== z07 ======
vrh AND wts mcs
y06 AND x06 jwh
mcs OR jwh swp
x07 XOR y07 fcs
swp XOR fcs z07
====== z08 ======
fcs AND swp jdb
y07 AND x07 btb
jdb OR btb kpt
x08 XOR y08 prr
kpt XOR prr z08
====== z09 ======
y09 XOR x09 kmk
x08 AND y08 mkj
kpt AND prr rdt
mkj OR rdt qvw
kmk XOR qvw z09
====== z10 ======
y10 XOR x10 wvn
qvw AND kmk qfq
y09 AND x09 spq
qfq OR spq trw
wvn XOR trw z10
====== z11 ======
x10 AND y

In [30]:
",".join(sorted(wire for wires in [("z31", "hkh"), ("z27", "bfq"), ("bng", "fjp"), ("hmt", "z18")] for wire in wires))

'bfq,bng,fjp,hkh,hmt,z18,z27,z31'

#### 25.1

In [32]:
key_lock_txt = open("inputs/25.txt").read()

In [ ]:
def analyze_key_lock(grid_txt):
    sizes = np.sum(np.array(list(map(list, grid_txt.split()))) == "#", axis=0) - 1
    is_key = grid_txt[0] == "."
    return is_key, sizes


def key_lock_combinations(key_lock_txt):
    keys = []
    locks = []
    for is_key, sizes in map(analyze_key_lock, key_lock_txt[:-1].split("\n\n")):
        if is_key:
            keys.append(sizes)
        else:
            locks.append(sizes)

    return sum(((key + lock) < 6).all() for key in keys for lock in locks)


key_lock_combinations(key_lock_txt)

np.int64(2618)

#### 25.2